# Finding base station neighbours

We will create a graph based on the $k$-NN method and then apply the enhanced criteria.

## Libraries importation

In [ ]:
# Please use these lines in every notebook you create

import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of `maps` (which is `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add `src` to the module search path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from global_variables import *
from python_scripts.miscellaneaous.data_processing import extract_data

In [ ]:
import pandas as pd

from python_scripts.graphs.graphs_creation import kNN_graph
from python_scripts.neighbours_criteria.enhanced_criteria import distance_criterion_enhanced, angle_criterion_enhanced, quadrant_criterion_enhanced
from python_scripts.city.city_utils import mean_distance_to_NN
from python_scripts.ihm.plots import plot_graph
from python_scripts.ihm.maps.map_neighbours import create_method_illustation_map

## Database import and data extraction

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=",")
df = extract_data(df, provider=PROVIDER, techno=TECHNO, region=REGION, min_info=True)
df.head()

## $k$-NN graph creation

In [ ]:
K = 15

In [ ]:
kNN_G, pos = kNN_graph(df, k=K)
print(len(kNN_G.edges))
plot_graph(kNN_G, pos)

## City detection

In [ ]:
mean_distances = mean_distance_to_NN(df[['x', 'y']], n_neighbours=N_NEIGH)

## Application of the criteria

In [ ]:
G_dis = distance_criterion_enhanced(kNN_G, pos, MEAN_DISTANCE_PARAMS, mean_distances)
print(len(G_dis.edges))
plot_graph(G_dis, pos)

In [ ]:
G_ang = angle_criterion_enhanced(kNN_G, pos, MEAN_DISTANCE_PARAMS, mean_distances)
print(len(G_ang.edges))
plot_graph(G_ang, pos)

In [ ]:
G_qua = quadrant_criterion_enhanced(kNN_G, pos, MEAN_DISTANCE_PARAMS, mean_distances)
print(len(G_qua.edges))
plot_graph(G_qua, pos)

In [ ]:
G_tot = distance_criterion_enhanced(kNN_G, pos, MEAN_DISTANCE_PARAMS, mean_distances)
G_tot = angle_criterion_enhanced(G_tot, pos, MEAN_DISTANCE_PARAMS, mean_distances)
G_tot = quadrant_criterion_enhanced(G_tot, pos, MEAN_DISTANCE_PARAMS, mean_distances)
print(len(G_tot.edges))
plot_graph(G_tot, pos)

## On a map

In [ ]:
create_method_illustation_map(df=df, base_graph=kNN_G, base_graph_name=f'{K}-NN graph', neigh_graph=G_tot, mean_distances=mean_distances, mean_distance_params=MEAN_DISTANCE_PARAMS, save_as=f'{K}-NN&enhanced-criteria', dis_filt=G_dis, ang_filt=G_ang, qua_filt=G_qua)